In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import t
import matplotlib.pyplot as plt
from arch import arch_model

In [2]:
np.random.seed(11)
ticker = "TRP.RO"
start_date = "2021-01-01"
confidence = 0.99
try:
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date)
except Exception as e:
    print("Error downloading", e)
    df = None
if df is None or df.empty:
    raise ValueError(f"No data returned for {ticker}")
if "Adj Close" in df.columns:
    df = df[["Adj Close"]].rename(columns={"Adj Close": "stock"})
else:
    df = df[["Close"]].rename(columns={"Close": "stock"})

In [3]:
prices_log = np.log(1 + df["stock"].pct_change())
prices_log.replace([np.inf, -np.inf], np.nan, inplace=True)
prices_log.dropna(inplace=True)
current_price = float(df["stock"].iloc[-1])
r = prices_log * 100
print(df)
print(r)

                              stock
Date                               
2021-01-04 00:00:00+02:00  0.235726
2021-01-05 00:00:00+02:00  0.234644
2021-01-06 00:00:00+02:00  0.237348
2021-01-07 00:00:00+02:00  0.238970
2021-01-08 00:00:00+02:00  0.238429
...                             ...
2026-01-22 00:00:00+02:00  0.471000
2026-01-23 00:00:00+02:00  0.468000
2026-01-26 00:00:00+02:00  0.470500
2026-01-27 00:00:00+02:00  0.492500
2026-01-28 00:00:00+02:00  0.527000

[1263 rows x 1 columns]
Date
2021-01-05 00:00:00+02:00   -0.460002
2021-01-06 00:00:00+02:00    1.145717
2021-01-07 00:00:00+02:00    0.681032
2021-01-08 00:00:00+02:00   -0.226627
2021-01-11 00:00:00+02:00   -0.454405
                               ...   
2026-01-22 00:00:00+02:00    2.580787
2026-01-23 00:00:00+02:00   -0.638978
2026-01-26 00:00:00+02:00    0.532766
2026-01-27 00:00:00+02:00    4.569853
2026-01-28 00:00:00+02:00    6.770609
Name: stock, Length: 1262, dtype: float64


In [4]:
model = arch_model(r, vol='Garch', p=1, q=1, dist='t', mean='Constant')
res = model.fit(disp='off')
forecasts = res.forecast(horizon=1)

In [5]:
next_mean = forecasts.mean.iloc[-1].values[0] / 100
next_var = forecasts.variance.iloc[-1].values[0] / (100**2)
next_sigma = np.sqrt(next_var)
nu = res.params['nu'] # Degrees of freedom

In [6]:
alpha = 1 - confidence
t_cutoff = t.ppf(alpha, df=nu)
var_log_return = -(next_mean + next_sigma * t_cutoff)
garch_var_price = current_price * (1 - np.exp(-var_log_return))
garch_var_pct = (1 - np.exp(-var_log_return)) * 100

In [8]:
print(f"\nTicker: {ticker}")
print(f"Last price: {current_price:.4f}")
print(f"GARCH(1,1) Student-t Volatility (Next Day): {next_sigma*100:.4f}%")
print(f"Estimated Degrees of Freedom (nu): {nu:.4f}")
print(f"GARCH Student-t VaR ({confidence*100:.0f}%): {garch_var_price:.4f} RON")
print(f"On {confidence*100:.0f}% of days, you should expect not to lose more than {garch_var_pct:.4f}% in a day")


Ticker: TRP.RO
Last price: 0.5270
GARCH(1,1) Student-t Volatility (Next Day): 4.6419%
Estimated Degrees of Freedom (nu): 3.0443
GARCH Student-t VaR (99%): 0.0995 RON
On 99% of days, you should expect not to lose more than 18.8826% in a day
